Easy to use Python API, for detail see

https://tweepy.readthedocs.io/en/latest/index.html


In [1]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
import datetime

In [2]:
spark = SparkSession \
          .builder \
          .config("spark.master", "local") \
          .appName("interfacing spark sql to hive metastore with no configuration file") \
          .config("hive.metastore.uris", "thrift://10.0.0.46:9083") \
          .enableHiveSupport() \
          .getOrCreate()

In [3]:
sc=spark.sparkContext

In [4]:
sc.setLogLevel("ERROR")

In [5]:
spark.sql("CREATE TABLE IF NOT EXISTS tweets (datetime STRING, text STRING) USING hive")

DataFrame[]

Now start streaming with tweepy

In [6]:
CONSUMER_KEY = 'your twitter consumer key'
CONSUMER_SECRET = 'your twitter consumer secret'
ACCESS_TOKEN = 'your twitter access token'
ACCESS_SECRET = 'your twitter access secret'

In [7]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import json, time, sys

import tweepy
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_SECRET)

class MyListener(StreamListener):
    def __init__(self, api=None):
        super().__init__()

    def on_status(self, status):
        text = status.text
        created = str(status.created_at)
        record = {'Text': text, 'Created At': created}
#Save to HIVE table tweets        
        filtered_data="".join(list(filter(lambda x: x!="@" and x!="'" and x!='"' \
                                          and x!="\n" and x!='{' and x!='}' and x!='(' and x!=')',text)))
        print(filtered_data)
        spark.sql("INSERT INTO tweets values ('"+created+"','"+filtered_data+"')")

    def on_error(self, status):
        print ('Something wrong with status', status)

    def on_limit(self, status):
        print ('Over the threshold', status)

    def on_timeout(self, status):
        print ('Timeout...')


stream = Stream(auth=auth, listener=MyListener())
stream.filter(track=['stock market'])

RT ACTBrigitte: Wages are up! Jobs are up! Stock market is soaring! I blame President Donald J Trump!
RT AZachParkinson: The S&amp;P 500 has returned more than 50% since Trump was elected, more than double the 23% average market return of pres…
RT ShriStocktipss: https://t.co/z8JWLkaSvt -Good Morning-#Premarket View for #Stock Market -27-Dec-2019 FridayNIFTY Weekly 23rd Dec-2…
RT Weinsteinlaw: 1 Almost half of Americans don’t own stocks2 10% of Americans own 86% of all stocks3 The stock market is not the eco…
RT AZachParkinson: The S&amp;P 500 has returned more than 50% since Trump was elected, more than double the 23% average market return of pres…
Trump stock market rally is far outpacing past US presidents https://t.co/KEh23K1npv
RT NormOrnstein: Obama stock market better than Trump. Please don’t retweet, it might upset Trump. https://t.co/NCbhOlIb6S
RT AZachParkinson: The S&amp;P 500 has returned more than 50% since Trump was elected, more than double the 23% average market return 

KeyboardInterrupt: 